In [162]:
def read_json(file_path):
    import json
    ds = []
   # urls = []

    for line in open(file_path, 'r'):
        data = json.loads(line)
        #urls.append(data["article_link"])
        ds.append([ data["headline"], int(data["is_sarcastic"]) ])
    return ds

In [163]:
import numpy as np

In [164]:
ds = read_json("./Datsets/news_dataset.nosync/Sarcasm_Headlines_Dataset_v2.json")
ds = np.array(ds, dtype=object )
ds = np.transpose(ds)
print(type(ds[1][0]))




In [165]:

from sklearn.model_selection import train_test_split

def split(data, test_percentage, transpose=True, val_dataset=True, val_percentage=0.2):

    df_train, df_test = train_test_split(data, test_size=test_percentage, random_state=42)
    
    if val_dataset==True:
        df_train, df_val = train_test_split(df_train, test_size=val_percentage, random_state=42)
        df_val = np.array(df_val, dtype=object )
    else:
        df_val=[]
    
    df_train = np.array(df_train, dtype=object )
    df_test = np.array(df_test, dtype=object )
    
    if(transpose == True):
        df_train = np.transpose(df_train)
        df_test = np.transpose(df_test)
        df_val = np.transpose(df_val)

    
    print(f" Training shape={df_train.shape} | Test shape={df_test.shape} | Validation Shape={df_val.shape}")
    return df_train, df_test, df_val


 Training shape=(2, 640) | Test shape=(2, 200) | Validation Shape=(2, 160)


In [167]:
from torch.utils.data import Dataset
import torch

class DataSet(Dataset):
    
    def __init__(self, text, label, tokenizer, max_len):
        self.text = text
        self.label = label
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.data = dict()
        
    def __len__(self):
        return len(self.text)
    

    def __getitem__(self, idx):
      text = str(self.text[idx])
      label = self.label[idx]
      encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_len,
        padding = False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
      )

      return {
        'texts': text,
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labels': torch.tensor(label, dtype=torch.long)
      }

      



In [168]:

def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = DataSet(text=df[0],label=df[1], tokenizer=tokenizer, max_len=max_len)
    return ds
  
BATCH_SIZE = 16
MAX_LEN = 128
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

train_data_loader[22]

{'texts': 'protesters clash with police during march to remember alton sterling',
 'input_ids': tensor([  101, 14339, 18751,  1114,  2021,  1219,  8943,  1106,  2676, 17138,
          1179,   188,  2083,  1979,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,

NOW WE IMPORT THE MODELS TO TEST AND RETRAIN

In [169]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AdamW, get_linear_schedule_with_warmup
from torch import nn
from collections import defaultdict

class_names = ['Sarcastic', 'Genuine']
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels = 2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /Users/salimhuneidi/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/re

In [170]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [171]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [172]:
vars(train_data_loader)


{'text': array(["chris hemsworth's wife elsa pataky posts sweet photo to celebrate his birthday",
        'opponents of peace',
        'egyptian woman wishes screaming protester husband would go bonkers for her once in a while',
        'zoo posting hourly updates on aphid about to give birth',
        "fda confirms psilocybin reduces risk of mindlessly following society's rules like fucking lemming",
        'single woman would love to hear them call her lonely now that she has basil plant',
        'banksy returns to new york city with one of his trademark rats',
        '11 doodles to help you hang in there after heartbreak',
        "father of muslim american war hero to trump: 'you have sacrificed nothing'",
        'the painful price of aging in prison',
        '12 publicists dead, 43 injured in struggle to transform the rock into dwayne johnson',
        'saudis snub obama on riyadh arrival amid growing tensions',
        'from behind the screen, you came so close',
        'l

In [173]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_loader,
    eval_dataset=val_data_loader,
    compute_metrics=compute_metrics,
)

In [174]:
trainer.train()

/Users/salimhuneidi/Desktop/Task Detector/TaskDetector/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 640
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 240













KeyboardInterrupt: 

In [ ]:
## TO retrive info 

pt_output = model(torch.tensor(tokenizer("Hello")["input_ids"]).reshape(1,3))

In [ ]:
pt_pred = nn.functional.softmax(pt_output.logits,dim=1)